# Currency Exchange Rates with Bayesian

Priscilla Chen (Calvin University)

## Main Question:

How do Differentials in Interest rates, Inflation, and Country’s economical Growth impact the currency exchange rate between US dollars and Chinese Yuan?

How confidently can we predict currency exchange rate from Differentials in Interest rates, Inflation, and Country’s Economical Growth?

*I would like to do a prediction model if possible, but if not, I’ll be satisfied with an inference model as well.*

## Data Preperation

In [ ]:
library(dagitty)


Attaching package: 'ggdag'

The following object is masked from 'package:stats':

    filter

Loading required package: dplyr


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

...

The following objects are masked from 'package:dplyr':

    count, do, tally

The following objects are masked from 'package:stats':

    binom.test, cor, cor.test, cov, fivenum, IQR, median, prop.test,
    quantile, sd, t.test, var

The following objects are masked from 'package:base':

    max, mean, min, prod, range, sample, sum

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.4     ✔ tidyr     1.3.1
✔ readr     2.1.5     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_factor() masks scales::col_factor()
✖ mosaic::count()     masks dplyr::count()
✖ purrr::cross()      masks mosaic::cross()
✖ purrr::discard()    masks scales::discard()
✖ mosaic::do()        masks dplyr::do()
✖ tidyr::expand()     masks Matrix::expand()
✖ dplyr::filter()     masks ggdag::filter(), stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ tidyr::pack()       masks Matrix::pack()
✖ mosaic::stat()      masks ggplot2::stat()
✖ mosaic::tally()     masks dplyr::tally()
✖ tidyr::unpack()     masks Matrix::unpack()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Data file 1: Monthly Exchange Rate between US dollars and Chinese Yuan, extracted from [Federal Researve](https://www.federalreserve.gov/releases/h10/current/) (Data range from 1990-2 to 2025-2)

In [ ]:
Exchange_rate <- read.csv("C:/Users/prisc/OneDrive - Calvin University/2Spring/Stat343/Currency-Exchange-Rate-Project/datafiles/FRB_H10 (1).csv", skip = 5)
Exchange_rate <- Exchange_rate |>
  mutate(Time.Period = paste0(Time.Period, "-01"))|>
  rename(Exchange_rate = RXI_N.M.CH,
         observation_date = Time.Period)

Data File 2: Inflation: YEARLY Consumer Prices for USA from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/FPCPITOTLZGUSA). (Note this dataset only contain data till 2023)

In [ ]:
Inflation_US <- read.csv("C:/Users/prisc/OneDrive - Calvin University/2Spring/Stat343/Currency-Exchange-Rate-Project/datafiles/FPCPITOTLZGUSA.csv")
Inflation_US <- Inflation_US |>
  rename(Inflation_US = FPCPITOTLZGUSA )

Data File 3: Inflation: YEARLY Consumer Prices for China from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/FPCPITOTLZGCHN). (Note this dataset only contain data till 2023)

In [ ]:
Inflation_CN <- read.csv("C:/Users/prisc/OneDrive - Calvin University/2Spring/Stat343/Currency-Exchange-Rate-Project/datafiles/FPCPITOTLZGCHN.csv")
Inflation_CN <- Inflation_CN |>
  rename(Inflation_CN = FPCPITOTLZGCHN )

Data File 4: Monthly GDP Ratio to Trend for USA, downloaded from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/USALORSGPRTSTSAM). (Note this dataset only contain data till 2023-11-01)

In [ ]:
GDP_US <- read.csv("C:/Users/prisc/OneDrive - Calvin University/2Spring/Stat343/Currency-Exchange-Rate-Project/datafiles/USALORSGPNOSTSAM.csv")
GDP_US <- GDP_US |>
  rename(GDP_US = USALORSGPNOSTSAM )

Data File 5: Monthly GDP Ratio to Trend for China, downloaded from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/CHNLORSGPRTSTSAM). (Note this dataset only contain data till 2024-02-01)

In [ ]:
GDP_CHINA <- read.csv("C:/Users/prisc/OneDrive - Calvin University/2Spring/Stat343/Currency-Exchange-Rate-Project/datafiles/CHNLORSGPNOSTSAM.csv")
GDP_CHINA <- GDP_CHINA |>
  rename(GDP_CN = CHNLORSGPNOSTSAM )

Data file 6: Monthly US Interest Rate (Rates which Federal Reserve charge on banks for overnight borrow), downloaded from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/INTDSRUSM193N). (Note this dataset only contain data till 2021-08-01)

In [ ]:
Interest_US <- read.csv("C:/Users/prisc/OneDrive - Calvin University/2Spring/Stat343/Currency-Exchange-Rate-Project/datafiles/INTDSRUSM193N.csv")
Interest_US <- Interest_US |>
  rename(Interest_US = INTDSRUSM193N)

Data file 7: Monthly Chinese Interest Rate (Rates which Federal Reserve charge on banks for overnight borrow), downloaded from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/INTDSRCNM193N)

In [ ]:
Interest_CN <- read.csv("C:/Users/prisc/OneDrive - Calvin University/2Spring/Stat343/Currency-Exchange-Rate-Project/datafiles/INTDSRCNM193N.csv")
Interest_CN <- Interest_CN |>
  rename(Interest_CN = INTDSRCNM193N)

**Joined Data:**

In [ ]:
datasets <- list(Exchange_rate,GDP_CHINA,GDP_US, Inflation_CN, Inflation_US, Interest_CN, Interest_US)

data <- reduce(datasets, full_join, by = "observation_date")

data <- data|>
  mutate(Dif_GPD = GDP_CN - GDP_US,
         Dif_Interest_Rate = Interest_CN - Interest_US,
         Dif_Inflation_Rate = Inflation_CN - Inflation_US)
glimpse(data)

Rows: 937
Columns: 11
$ observation_date   <chr> "1990-01-01", "1990-02-01", "1990-03-01", "1990-04-…
$ Exchange_rate      <dbl> 4.7339, 4.7339, 4.7339, 4.7339, 4.7339, 4.7339, 4.7…
$ GDP_CN             <dbl> 96.86580, 96.60166, 96.44037, 96.36356, 96.35556, 9…
$ GDP_US             <dbl> 101.34920, 101.37080, 101.36490, 101.31980, 101.227…
$ Inflation_CN       <dbl> 3.052290, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ Inflation_US       <dbl> 5.397956, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ Interest_CN        <dbl> NA, NA, 9.00, 9.00, 9.00, 9.00, 9.00, 7.92, 7.92, 7…
$ Interest_US        <dbl> 7.00, 7.00, 7.00, 7.00, 7.00, 7.00, 7.00, 7.00, 7.0…
$ Dif_GPD            <dbl> -4.48340, -4.76914, -4.92453, -4.95624, -4.87174, -…
$ Dif_Interest_Rate  <dbl> NA, NA, 2.00, 2.00, 2.00, 2.00, 2.00, 0.92, 0.92, 0…
$ Dif_Inflation_Rate <dbl> -2.3456663, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…

*Alternatively, I want to also scale these variables (those starting with Dif), which I don’t know how to yet.*

## Casual Diagram:

[Economics Help](https://www.economicshelp.org/macroeconomics/exchangerate/factors-influencing/) listed 5 factors that would affect exchange rate: rate of growth, Interest rate, Inflation rate, Competitiveness, and Speculations. Since competitiveness and Speculations are hard to measure quantitatively, they are not included. Besides, rate of growth is measured by GDP ratio.

*The code for Casual Diagram is producing error right not, I haven’t got to fixing it yet*

In [ ]:
# causal_diagram <- dagitty("dag{
#   Interest_Rate -> Exchange_Rate;
#   Interest_Rate -> Inflation_Rate;
#   Inflation_Rate -> Exchange_Rate;
#   Growth_Rate -> Exchange_Rate;
# }")
# gg_dag(causal_diagram)